In [6]:
import sys
sys.path.append('..')

from pipeline_objects.data_loader import DataLoader
from pipeline_objects.preprocessor import DataPreprocessor


In [2]:
loader = DataLoader('../Data', ['open_results'])
data = loader.load()

/Users/hassan/Documents/wod-prediction/WOD-prediction/examples/../pipeline_objects/data_loader.py:21: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
/Users/hassan/Documents/wod-prediction/WOD-prediction/examples/../pipeline_objects/data_loader.py:21: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [5]:
preprocessing_config = {
    'open_results':
    {
        'melt': False,
        'create_description_embeddings' : False,
    },
    'workout_descriptions':{},
    'benchmark_stats':{},
    'athlete_info':{}
}
preprocessor = DataPreprocessor(preprocessing_config)
preprocessor.transform(data)


ValueError: Both open results and workout descriptions must be provided to transform open results